In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import scipy as sp
import seaborn as sns
from scipy import signal
from statsmodels.tsa.stattools import adfuller
from IPython.display import Markdown as md
import copy

# Fourier Histograms for the Individuals Who Drank More Than 4 Times

In [2]:
datafile = "/Users/djpassey/Data/Muri/SHINE_EMA_Round1_19May2020.csv"
ema = pd.read_csv(datafile, parse_dates=True)
# Convert notification time to a datetime object
ema['Notification.Time'] = pd.to_datetime(ema['Notification.Time'])
# 'Num_Alcohol is our new column containing the drinking data we want to measure. In this notebook
# we are interested in 'if' the individual drank or not so we simply set the column equal to "Had_Alcohol"
# rather than summing up the number of drinks of each type the individual reported
ema['Num_Alcohol'] = ema.HadAlcohol

In [3]:
# Constants for determining if a prompt occured in the morning or evening
MORNING = ['FirstMorning', 'Morning'] 
EVENING = ['Evening']
# p-value for this notebook
PVAL = 0.01
# Notebook color scheme
COLORS = ["salmon", "teal", "grey", "green"]

def drink_sessions(df, prompt=None):
    """ Function for removing data from prompts where drinking was not assessed
    
        Parameters
        ----------
        df (pandas Dataframe): ema data
        prompt (string): one of `["morning", "evening", None]`. Specifies if the 
        function should return only data from morning or evening prompts. Defaults 
        to None and therefore returns data from both morning and evening prompts
        
        Returns
        -------
        ds (pandas Dataframe): data frame containing only prompts where drinking was 
        assessed. If `prompt` was not none, the data only contains information from  
        either morning or evening prompts. The dataframe is also sorted by notification time
    """
    if prompt is None:
        ds = df[df["Session.Name"].isin(MORNING+EVENING)]
    if prompt is "morning":
        ds = df[df["Session.Name"].isin(MORNING)]
    if prompt is "evening":
        ds = df[df["Session.Name"].isin(EVENING)]    
    ds.fillna({"Num_Alcohol":0, "HadAlcohol":0})
    ds.sort_values("Notification.Time", inplace=True)
    return ds

def fourier_transform(t, y):
    """ Take the fourier transform of a signal, rescale and provide a frequency axis
        array for easy plotting
        
        Parameters
        ----------
        t (array): Evenly spaced time values
        y (array): Time series, `y[i]` corresponds to the value of the 
        time series at time `t[i]`
    """
    total_time = float(t[-1] - t[0])
    N = len(y)
    xf = np.arange(N)/ (total_time)
    yf = sp.fft.fft(y) / N
    # Take half of the dfft and multiply by 2 because it is a mirror image
    yf = 2*np.abs(yf[:N//2])
    xf = xf[:N//2]
    return xf, yf

def hist_templ(
    *data, 
    xlab="Value", 
    ylab="Frequency", 
    title="Histogram", 
    label=["Morning Prompt", "Evening Prompt"],
    bins=20,
    alpha=0.6
):
    """Histogram template function"""
    for i, x in enumerate(data):
        plt.hist(x, bins=bins, color=COLORS[i], alpha=0.6, label=label[i])
    plt.legend()
    plt.ylabel(ylab)
    plt.xlabel(xlab)
    p = plt.title(title)
    return p

# Extract Mood and Drinking Time Series

We separate by morning and evening prompt so that there are 24 hours between each datapoint. The dictionary `processed_data` contains all of the processed data that we study in this notebook. Here is a summary of the entries in the `processed_data` dictionary. (Some of these entries are added later

1. `processesed_data["id"]` is a 1D array of id numbers. All of the following arrays are kept in order corresponding to id number.

2. `processesed_data["drink.morning"]` is a 2D array where the `i`th row is a drinking time series corresponding to the individual with id number equal to `processesed_data["id"][i]`. Each drinking time series here has 28 entries corresponding to the 28 *morning* prompts.

3. `processesed_data["drink.evening"]` is a 2D array where the `i`th row is a drinking time series corresponding to the individual with id number equal to `processesed_data["id"][i]`. Each drinking time series here has 28 entries corresponding to the 28 *evening* prompts.

4. `processesed_data["mood.morning"]` is a 2D array where the `i`th row is a positive mood time series corresponding to the individual with id number equal to `processesed_data["id"][i]`. Each mood time series here has 28 entries corresponding to the 28 *morning* prompts.

5. `processesed_data["mood.evening"]` is a 2D array where the `i`th row is a positive mood time series corresponding to the individual with id number equal to `processesed_data["id"][i]`. Each mood time series here has 28 entries corresponding to the 28 *morning* prompts.

6. `processed_data["{mood or drink}.{morning or evening}.ft"]` For entries 2-5 above, appending `".ft"` onto the end of the key (e.g. `"mood.evening" -> "mood.evening.ft")` produces a 2D array where each row is the fourier transform of the corresponding row in the original time series array. (e.g. `processesed_data["mood.evening.ft"][2, :]` contains the fourier transform of `processesed_data["mood.evening"][2, :]`)

7. `processed_data["{mood or drink}.{morning or evening}.adfuller"]` For entries 2-5 above, appending `".adfuller"` onto the end of the key (e.g. `"mood.evening" -> "mood.evening.adfuller")` produces a 1D array where the `i`th entry is a p-value corresponding to an Augmented Dickey-Fuller unit root test (a statistical test for stationarity) applied to the `i`th row of the corresponding time series array. (e.g. `processesed_data["mood.evening.ft"][2]` contains the p-value corresponding to a Augmented Dickey-Fuller test run on `processesed_data["mood.evening"][2, :]`)



In [4]:
processed_data = {
    "id" : [],
    "drink.morning" : [],
    "drink.evening" : [],
    "mood.morning" : [],
    "mood.evening" : []
}

time_series_keys = ["drink.morning", "drink.evening", "mood.morning", "mood.evening"]

# Separate morning and evening prompts
morn = drink_sessions(ema, prompt="morning")
eve = drink_sessions(ema, prompt="evening")

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
# Participants to exclude
EXCLUDE_ID = [52927]

# Minimum number of times an individual must drink
DRINK_MIN = 4

# Drinking and Mood data tuples
drinkmorn = tuple()
drinkeve = tuple()
moodmorn = tuple()
moodeve = tuple()

for idnum in ema.ID.unique():
    if idnum not in EXCLUDE_ID:
        # Look at subset of data corresponding to the current ID
        morn_id = morn[morn.ID == idnum]
        eve_id = eve[eve.ID == idnum]
        morning_drink = morn_id["Num_Alcohol"].fillna(0).values
        if sum(morning_drink) > DRINK_MIN:
            # add ID numbers
            processed_data["id"].append(idnum)
            # Extract morning and evening drinks time series and append it to tuple
            drinkmorn += (morn_id["Num_Alcohol"].fillna(0).values,)
            drinkeve += (eve_id["Num_Alcohol"].fillna(0).values,)
            # Extract morning and eveing positive mood time series
            morn_m = morn_id.PositiveMood
            eve_m = eve_id.PositiveMood
            # Fill nans with the mean and append to tuple
            moodmorn += (morn_m.fillna(np.mean(morn_m)).values,)
            moodeve += (eve_m.fillna(np.mean(eve_m)).values,)
    
# Turn tuple of timeseries into arrays
processed_data["drink.morning"] = np.vstack(drinkmorn)
processed_data["drink.evening"] = np.vstack(drinkeve)
processed_data["mood.morning"] = np.vstack(moodmorn)
processed_data["mood.evening"] = np.vstack(moodeve)

In [6]:
n = processed_data["drink.morning"].shape[0]
print(f"{n} Individuals drank more than {DRINK_MIN} times during the study")

53 Individuals drank more than 4 times during the study


### Fourier Transform

In [7]:
# Take fourier transform of each time series type
for key in time_series_keys:
    Yf = tuple()
    for ts in processed_data[key]:
        x = np.arange(28)
        xf, yf = fourier_transform(x, ts)
        Yf += (yf,)
    processed_data[key + ".ft"] = np.vstack(Yf)

### Stationarity Test

In [8]:
with np.errstate(divide='ignore'):
    for key in time_series_keys:
        ts = processed_data[key]
        pvals = [adfuller(x)[1] for x in ts]
        processed_data[key + ".adfuller"] = np.array(pvals)

### Stationarity Table

In [11]:
pval = 0.01
# Remove non drinkers
drink_station = processed_data["drink.morning.adfuller"]
mood_station = processed_data["mood.morning.adfuller"]

sta_mood_sta_drink = sum((drink_station < pval) * (mood_station < pval) )
sta_mood_not_drink = sum((drink_station > pval) * (mood_station < pval) )
not_mood_sta_drink = sum((drink_station < pval) * (mood_station > pval) )
not_mood_not_drink = sum((drink_station > pval) * (mood_station > pval) )

stationarity_table = f"""#### Morning Mood and Drinking Stationarity

|  | Stationary Drinking | Non-Stationary Drinking | Totals |
| --- | --- | --- | --- |
| **Stationary Mood**|{sta_mood_sta_drink} |{sta_mood_not_drink} | {sta_mood_sta_drink + sta_mood_not_drink}
| **Non Stationary Mood**| {not_mood_sta_drink}| {not_mood_not_drink}| {not_mood_sta_drink + not_mood_not_drink}
| **Totals**| {sta_mood_sta_drink + not_mood_sta_drink}| {sta_mood_not_drink + not_mood_not_drink} | {sta_mood_sta_drink + sta_mood_not_drink + not_mood_sta_drink + not_mood_not_drink}

"""
md(stationarity_table)

#### Morning Mood and Drinking Stationarity

|  | Stationary Drinking | Non-Stationary Drinking | Totals |
| --- | --- | --- | --- |
| **Stationary Mood**|18 |11 | 29
| **Non Stationary Mood**| 15| 9| 24
| **Totals**| 33| 20 | 53



In [22]:
ids = processed_data["id"]
ema_drink_gtr_4 = drink_sessions(ema)
ema_drink_gtr_4 = ema_drink_gtr_4[ema_drink_gtr_4.ID.isin(ids)]

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [24]:
ema_drink_gtr_4.value_counts("Condition")

Condition
mindful        1176
control        1008
perspective     784
dtype: int64

In [26]:

for d in processed_data["drink.morning"]:
    plt.plot(d, "o")
plt.xlabel("Day")

['SHINEID',
 'ID',
 'Notification.Time',
 'Session.Name',
 'Notification.No',
 'LifePak.Download.No',
 'Responded',
 'Completed.Session',
 'Session.Instance',
 'Session.Instance.Response.Lapse',
 'Session.Length',
 'Reminders.Delivered',
 'PositiveMood',
 'NegativeMood',
 'HadAlcohol',
 'Alcohol_Alone',
 'Alcohol_Friend',
 'Alcohol_Coworker',
 'Alcohol_Family',
 'Alcohol_Stranger',
 'Alcohol_SigOther',
 'Num_Beer',
 'Num_Wine',
 'Num_Liquor',
 'Alcohol_Size_None',
 'Alcohol_Size_One',
 'Alcohol_Size_Small',
 'Alcohol_Size_Large',
 'Excer_Vigor',
 'Excer_Moder',
 'Excer_Mild',
 'Num_Caffeine',
 'Num_Water',
 'Craving_Alc',
 'Converse_Alc',
 'Converse_Binge',
 'Converse_Drunk',
 'Converse_Alc_None',
 'Alc_Converse_Valence',
 'Binge_Converse_Valence',
 'Drunk_Converse_Valence',
 'Converse_Water',
 'Converse_Excer',
 'Converse_Caffeine',
 'Converse_None',
 'EmoReg_None',
 'EmoReg_Reappraisal',
 'EmoReg_Suppression',
 'EmoReg_Avoidance',
 'EmoReg_Acceptance',
 'EmoReg_SocialSupport',
 'EmoR